In [94]:
import requests
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


In [95]:
STATIC_JSON_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json"

resp = requests.get(STATIC_JSON_URL)
resp.raise_for_status()
raw = resp.json()                        # list of launches (dicts)
data = pd.json_normalize(raw)            # flatten nested JSON into columns

data.head(3)


,static_fire_date_utc,static_fire_date_unix,tbd,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,auto_update,failures,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,False,0.0,5e9d0d95eda69955f709d1eb,False,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,True,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png,https://images2.imgbox.com/40/e3/GypSkayF_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,False,0.0,5e9d0d95eda69955f709d1eb,False,"Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,True,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]",2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/4f/e3/I0lkuJ2e_o.png,https://images2.imgbox.com/be/e7/iNqsqVYM_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,False,0.0,5e9d0d95eda69955f709d1eb,False,Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,True,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/3d/86/cnu0pan8_o.png,https://images2.imgbox.com/4b/bd/d8UxLh4q_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN


In [96]:
# look at the first few rows and include the static_fire_date_utc column
data[['static_fire_date_utc']].head()
pd.to_datetime(data.loc[0, 'static_fire_date_utc']).year



2006

In [97]:
# keep core features + identifiers for enrichment
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']].copy()

# remove rows with multiple cores/payloads (we only keep single-core/single-payload flights)
data = data[data['cores'].map(len) == 1]
data = data[data['payloads'].map(len) == 1]

# unwrap the single-element lists
data['cores']    = data['cores'].map(lambda L: L[0])      # now a dict like {'core': 'id', 'flight': ..., ...}
data['payloads'] = data['payloads'].map(lambda L: L[0])   # now a single payload id string

# convert date & filter to a fixed range (<= 2020-11-13)
data['date'] = pd.to_datetime(data['date_utc']).dt.date
data = data[data['date'] <= dt.date(2020, 11, 13)].reset_index(drop=True)

len(data), data.head(2)


(94,
                      rocket                  payloads  \
 0  5e9d0d95eda69955f709d1eb  5eb0e4b5b6c3bb0006eeb1e1   
 1  5e9d0d95eda69955f709d1eb  5eb0e4b6b6c3bb0006eeb1e2   
 
                   launchpad  \
 0  5e9e4502f5090995de566f86   
 1  5e9e4502f5090995de566f86   
 
                                                                                                                                                                                             cores  \
 0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
 1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
 
    flight_number                  date_utc        date  
 0              1  2006-03-24T22:30:00.000Z  2006-03-24  
 1       

In [98]:
BoosterVersion, PayloadMass, Orbit, LaunchSite = [], [], [], []
Outcome, Flights, GridFins, Reused, Legs, LandingPad   = [], [], [], [], [], []
Block, ReusedCount, Serial                              = [], [], []
Longitude, Latitude                                     = [], []


In [99]:
# simple response cache (in-memory)
_cache = {}

def _get(url):
    if url in _cache:
        return _cache[url]
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    j = r.json()
    _cache[url] = j
    return j

def getBoosterVersion(df):
    base = "https://api.spacexdata.com/v4/rockets/"
    for rocket_id in df['rocket']:
        if pd.isna(rocket_id):
            BoosterVersion.append(None)
            continue
        j = _get(base + str(rocket_id))
        BoosterVersion.append(j.get('name'))

def getLaunchSite(df):
    base = "https://api.spacexdata.com/v4/launchpads/"
    for lp_id in df['launchpad']:
        if pd.isna(lp_id):
            Longitude.append(None); Latitude.append(None); LaunchSite.append(None)
            continue
        j = _get(base + str(lp_id))
        Longitude.append(j.get('longitude'))
        Latitude.append(j.get('latitude'))
        LaunchSite.append(j.get('name'))

def getPayloadData(df):
    base = "https://api.spacexdata.com/v4/payloads/"
    for payload_id in df['payloads']:
        if pd.isna(payload_id):
            PayloadMass.append(None); Orbit.append(None)
            continue
        j = _get(base + str(payload_id))
        PayloadMass.append(j.get('mass_kg'))  # may be None
        Orbit.append(j.get('orbit'))

def getCoreData(df):
    base = "https://api.spacexdata.com/v4/cores/"
    for core_entry in df['cores']:
        # core_entry is a dict with keys like 'core', 'gridfins', 'reused', 'legs', 'landpad', 'landing_success', 'landing_type', 'flight'
        core_id = core_entry.get('core') if isinstance(core_entry, dict) else None

        if core_id is not None:
            j = _get(base + str(core_id))
            Block.append(j.get('block'))
            ReusedCount.append(j.get('reuse_count'))
            Serial.append(j.get('serial'))
        else:
            Block.append(None); ReusedCount.append(None); Serial.append(None)

        # attributes directly from the launch "cores" object
        Outcome.append(f"{core_entry.get('landing_success')} {core_entry.get('landing_type')}")
        Flights.append(core_entry.get('flight'))
        GridFins.append(core_entry.get('gridfins'))
        Reused.append(core_entry.get('reused'))
        Legs.append(core_entry.get('legs'))
        LandingPad.append(core_entry.get('landpad'))


In [100]:
getBoosterVersion(data)
getLaunchSite(data)
getPayloadData(data)
getCoreData(data)

# quick length sanity check (all should match #rows)
assert len(BoosterVersion) == len(data) == len(PayloadMass) == len(Orbit) == len(LaunchSite)


KeyboardInterrupt: 

In [ ]:
launch_dict = {
    'FlightNumber':   list(data['flight_number']),
    'Date':           list(data['date']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass':    PayloadMass,
    'Orbit':          Orbit,
    'LaunchSite':     LaunchSite,
    'Outcome':        Outcome,
    'Flights':        Flights,
    'GridFins':       GridFins,
    'Reused':         Reused,
    'Legs':           Legs,
    'LandingPad':     LandingPad,
    'Block':          Block,
    'ReusedCount':    ReusedCount,
    'Serial':         Serial,
    'Longitude':      Longitude,
    'Latitude':       Latitude,
}

df = pd.DataFrame(launch_dict)
df.head(5)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


In [ ]:
data_falcon9 = df[df['BoosterVersion'] != 'Falcon 1'].copy()

# reset FlightNumber to be 1..N
data_falcon9.loc[:, 'FlightNumber'] = range(1, len(data_falcon9) + 1)

data_falcon9.head(5)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


In [ ]:
data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].astype('float64')
pm_mean = data_falcon9['PayloadMass'].mean(skipna=True)

data_falcon9['PayloadMass'] = data_falcon9['PayloadMass'].replace(np.nan, pm_mean)

# Verify
data_falcon9.isnull().sum()


FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [ ]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)


In [ ]:
# What boosters are present?
data_falcon9['BoosterVersion'].value_counts().head(10)

# Outcomes overview
data_falcon9['Outcome'].value_counts().head(10)

# Numeric summary
data_falcon9[['PayloadMass','Flights','Block','ReusedCount','Longitude','Latitude']].describe()


,PayloadMass,Flights,Block,ReusedCount,Longitude,Latitude
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,6123.547647,1.788889,3.500000,3.188889,-86.366477,29.449963
std,4732.115291,1.213172,1.595288,4.194417,14.149518,2.141306
min,350.000000,1.000000,1.000000,0.000000,-120.610829,28.561857
25%,2510.750000,1.000000,2.000000,0.000000,-80.603956,28.561857
50%,4701.500000,1.000000,4.000000,1.000000,-80.577366,28.561857
75%,8912.750000,2.000000,5.000000,4.000000,-80.577366,28.608058
max,15600.000000,6.000000,5.000000,13.000000,-80.577366,34.632093


In [ ]:
pip install beautifulsoup4 requests pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# ---------------------------
# helper functions from lab
# ---------------------------
def date_time(table_cells):
    """Return [date, time] text from the first column."""
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """Return booster version text from the 2nd column."""
    out = ''.join([bv for i, bv in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

def landing_status(td):
    # keep everything like "Success (drone ship)" / "Failure (ground pad)" / "No attempt"
    return td.get_text(" ", strip=True)
#def landing_status(table_cells):
    """Return landing status from the last column."""
    out = [i for i in table_cells.strings][0]
    return out

def get_mass(table_cells):
    """Return payload mass like '15,600 kg' (or 0 if missing)."""
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        idx = mass.find("kg")
        new_mass = mass[: idx + 2] if idx != -1 else mass
    else:
        new_mass = 0
    return new_mass

def extract_column_from_header(row):
    """Clean header cell to get the column name."""
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    column_name = ' '.join(row.contents)
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/91.0.4472.124 Safari/537.36")
}

resp = requests.get(static_url, headers=headers, timeout=30)
resp.raise_for_status()                               # ensure 200 OK
soup = BeautifulSoup(resp.text, "html.parser")

# quick sanity check
print(soup.title.text)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [ ]:
# find all tables on the page
html_tables = soup.find_all("table")

# our target is the 3rd table on the page
first_launch_table = html_tables[2]
# print(first_launch_table)  # (optional) see raw HTML

# get column names from <th> in the target table
column_names = []
for th in first_launch_table.find_all("th"):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [ ]:
# start with the discovered columns
launch_dict = dict.fromkeys(column_names)

# remove the noisy 'Date and time ( )' header
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# initialize all lists we will fill
launch_dict['Flight No.']        = []
launch_dict['Launch site']       = []
launch_dict['Payload']           = []
launch_dict['Payload mass']      = []
launch_dict['Orbit']             = []
launch_dict['Customer']          = []
launch_dict['Launch outcome']    = []
# additional split columns
launch_dict['Version Booster']   = []
launch_dict['Booster landing']   = []
launch_dict['Date']              = []
launch_dict['Time']              = []


In [ ]:
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # determine if the row begins a launch entry (Flight No. is numeric)
        flag = False
        flight_number = None
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()

        row = rows.find_all('td')

        if flag and row:
            extracted_row += 1

            # --- Date & Time (first cell) ---
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',') if len(datatimelist) > 0 else None
            time = datatimelist[1] if len(datatimelist) > 1 else None

            # --- Booster version (second cell) ---
            bv = booster_version(row[1])
            if not bv:
                # fallback: sometimes the <a> holds the text
                bv = row[1].a.string if row[1].a and row[1].a.string else None

            # --- Launch site ---
            # --- Launch site ---
            launch_site = row[2].get_text(" ", strip=True)   # e.g., "CCAFS SLC-40", "KSC LC-39A", "VAFB SLC-4E"


            # --- Payload ---
            payload = (row[3].a.string if row[3].a and row[3].a.string
                       else row[3].get_text(strip=True))

            # --- Payload mass ---
            payload_mass = get_mass(row[4])

            # --- Orbit ---
            orbit = (row[5].a.string if row[5].a and row[5].a.string
                     else row[5].get_text(strip=True))

            # --- Customer ---
            customer = (row[6].a.string if row[6].a and row[6].a.string
                        else row[6].get_text(strip=True))

            # --- Launch outcome ---
            launch_outcome = list(row[7].strings)[0].strip() if row[7].strings else row[7].get_text(strip=True)

            # --- Booster landing ---
            booster_landing = landing_status(row[8]).strip() if len(row) > 8 else None

            # -------- append to dict --------
            launch_dict['Flight No.'].append(flight_number)
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)
            launch_dict['Version Booster'].append(bv)
            launch_dict['Launch site'].append(launch_site)
            launch_dict['Payload'].append(payload)
            launch_dict['Payload mass'].append(payload_mass)
            launch_dict['Orbit'].append(orbit)
            launch_dict['Customer'].append(customer)
            launch_dict['Launch outcome'].append(launch_outcome)
            launch_dict['Booster landing'].append(booster_landing)

print("Total rows extracted:", extracted_row)


Total rows extracted: 121


In [ ]:
df = pd.DataFrame({k: pd.Series(v) for k, v in launch_dict.items()})
print(df.shape)
df.head(10)


(121, 11)


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,"CCAFS , SLC-40",Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure [ 9 ] [ 10 ] (parachute),4 June 2010,18:45
1,2,"CCAFS , SLC-40",Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure [ 9 ] [ 14 ] (parachute),8 December 2010,15:43
2,3,"CCAFS , SLC-40",Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt,22 May 2012,07:44
3,4,"CCAFS , SLC-40",SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,"CCAFS , SLC-40",SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt,1 March 2013,15:10
5,6,"VAFB , SLC-4E",CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.17B10038,Uncontrolled (ocean) [ d ],29 September 2013,16:00
6,7,"CCAFS , SLC-40",SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,No attempt [ 38 ],3 December 2013,22:41
7,8,"CCAFS , SLC-40",Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,No attempt [ 43 ],6 January 2014,22:06
8,9,"Cape Canaveral , LC-40",SpaceX CRS-3,"2,296 kg",LEO,NASA,Success,F9 v1.1,Controlled (ocean) [ d ] [ 46 ],18 April 2014,19:25
9,10,"Cape Canaveral , LC-40",Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,Controlled (ocean) [ d ] [ 46 ],14 July 2014,15:15


In [ ]:
df.to_csv('spacex_web_scraped.csv', index=False)
print("Saved to spacex_web_scraped.csv")


Saved to spacex_web_scraped.csv


In [ ]:
def to_kg_number(s):
    if not s or s == 0:
        return None
    m = re.search(r'([\d,\.]+)\s*kg', s)
    if not m:
        return None
    return float(m.group(1).replace(',', ''))

df['Payload mass (kg)'] = df['Payload mass'].map(to_kg_number)
df[['Payload mass', 'Payload mass (kg)']].head(8)


,Payload mass,Payload mass (kg)
0,0,NaN
1,0,NaN
2,525 kg,525.0
3,"4,700 kg",4700.0
4,"4,877 kg",4877.0
5,500 kg,500.0
6,"3,170 kg",3170.0
7,"3,325 kg",3325.0


In [ ]:
list(df.columns)


['Flight No.',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Version Booster',
 'Booster landing',
 'Date',
 'Time',
 'Payload mass (kg)']

In [ ]:
data_falcon9 = df[df['Version Booster'] != 'Falcon 1'].copy()
len(data_falcon9)



121

In [ ]:
import datetime as dt
import pandas as pd
import requests

# Start from the raw API-normalized frame: `data`
api = data[['rocket','payloads','launchpad','cores','flight_number','date_utc']].copy()

# keep single-core/single-payload & date limit (as in the lab)
api = api[api['cores'].map(len) == 1]
api = api[api['payloads'].map(len) == 1]
api['date'] = pd.to_datetime(api['date_utc']).dt.date
api = api[api['date'] <= dt.date(2020, 11, 13)].reset_index(drop=True)

# map rocket id -> name (BoosterVersion)
rockets = pd.DataFrame(requests.get("https://api.spacexdata.com/v4/rockets").json())[['id','name']]
rockets = rockets.rename(columns={'id':'rocket','name':'BoosterVersion'})
api = api.merge(rockets, on='rocket', how='left')

# filter out Falcon 1 (keep Falcon 9)
data_falcon9 = api[api['BoosterVersion'] != 'Falcon 1'].copy()

len(data_falcon9)


0

In [ ]:
api['BoosterVersion'].value_counts(dropna=False)


Series([], Name: count, dtype: int64)

In [ ]:
# Create the 'Class' column from Booster landing
def to_class(s):
    t = str(s).strip().lower()
    # count only true successes as 1
    if 'success' in t and not any(x in t for x in ['partial','uncontrolled','failure']):
        return 1
    return 0

df['Class'] = df['Booster landing'].map(to_class).fillna(0).astype(int)

# Quick check
print(df[['Booster landing','Class']].head())


                    Booster landing  Class
0  Failure [ 9 ] [ 10 ] (parachute)      0
1  Failure [ 9 ] [ 14 ] (parachute)      0
2                        No attempt      0
3                        No attempt      0
4                        No attempt      0


In [ ]:
# 1) Quick sanity checks
assert 'Class' in df.columns, "Expected a column named 'Class' (1=landed, 0=not landed)"
df['Class'] = pd.to_numeric(df['Class'], errors='coerce')

# 2) Overall success rate
success_rate = df['Class'].mean()                 # fraction in [0,1]
print("Success rate (fraction):", success_rate)
print("Success rate (%):", round(success_rate * 100, 2), "%")

# 3) Helpful counts
total = len(df)
successes = int(df['Class'].sum())
failures = int(total - successes)
print(f"Total: {total} | Successes: {successes} | Failures: {failures}")

# (Optional) by landing outcome if you kept it
if 'Outcome' in df.columns:
    print(df['Outcome'].value_counts())

# 4) Export for the next lab
df.to_csv("dataset_part_2.csv", index=False)
print("Saved dataset_part_2.csv")


Success rate (fraction): 0.6611570247933884
Success rate (%): 66.12 %
Total: 121 | Successes: 80 | Failures: 41
Saved dataset_part_2.csv


In [ ]:
import pandas as pd
import numpy as np
import re

print("Columns:", df.columns.tolist())

# 1) Create Class if missing (1 = successful first-stage landing, 0 = otherwise)
if 'Class' not in df.columns:
    if 'Outcome' in df.columns:
        # API-enriched lab: Outcome like "True ASDS", "False Ocean", "None None"
        df['Class'] = df['Outcome'].astype(str).str.startswith('True').astype(int)

    elif 'landing_success' in df.columns:
        # If you kept the boolean from cores
        df['Class'] = df['landing_success'].astype(float).fillna(0).astype(int)

    elif 'Booster landing' in df.columns:
        # Web-scraped lab: strings like 'Success', 'Failure', 'Ocean', 'No attempt'...
        def to_class(s):
            t = str(s).strip().lower()
            # treat only explicit successes as 1
            if 'success' in t and not any(x in t for x in ['partial', 'uncontrolled', 'failure']):
                return 1
            return 0
        df['Class'] = df['Booster landing'].map(to_class).fillna(0).astype(int)

    elif 'cores' in df.columns:
        # If you still have the cores dict column from API
        def core_success(c):
            if isinstance(c, dict):
                return int(bool(c.get('landing_success')))
            return 0
        df['Class'] = df['cores'].map(core_success).astype(int)

    else:
        raise ValueError(
            "Couldn't infer landing success. Expected one of: "
            "'Outcome', 'landing_success', 'Booster landing', or 'cores'."
        )

# 2) Convert to numeric just in case, then compute success rate
df['Class'] = pd.to_numeric(df['Class'], errors='coerce').fillna(0).astype(int)

success_rate = df['Class'].mean()
print("Success rate (fraction):", success_rate)
print("Success rate (%):", round(success_rate * 100, 2), "%")
print("Total:", len(df), "| Successes:", int(df['Class'].sum()), "| Failures:", int(len(df) - df['Class'].sum()))

# 3) Save for the next lab
df.to_csv("dataset_part_2.csv", index=False)
print("Saved dataset_part_2.csv")


Columns: ['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time', 'Payload mass (kg)', 'Class']
Success rate (fraction): 0.6611570247933884
Success rate (%): 66.12 %
Total: 121 | Successes: 80 | Failures: 41
Saved dataset_part_2.csv


In [ ]:
# count number of launches at each site
site_counts = df['Launch site'].value_counts()

print(site_counts)


Launch site
CCAFS , SLC-40             39
KSC , LC-39A               32
Cape Canaveral , LC-40     19
VAFB , SLC-4E              16
CCSFS , SLC-40             10
Cape Canaveral , SLC-40     1
CCAFS SLC-40                1
CCSFS , SLC-40 [ 597 ]      1
KSC , LC-39A [ 661 ]        1
CCSFS , SLC-40 [ 665 ]      1
Name: count, dtype: int64


In [ ]:
df['Orbit'].value_counts()


Orbit
LEO            67
GTO            33
SSO             7
Polar           7
MEO             3
HEO             2
Polar orbit     1
Sub-orbital     1
Name: count, dtype: int64

In [ ]:
# Count total launches per orbit type
outcome_counts = df.groupby('Orbit')['Class'].value_counts().unstack(fill_value=0)

print(outcome_counts)


Class         0   1
Orbit              
GTO          15  18
HEO           1   1
LEO          18  49
MEO           1   2
Polar         3   4
Polar orbit   1   0
SSO           1   6
Sub-orbital   1   0


In [ ]:
success_rate_per_orbit = df.groupby('Orbit')['Class'].mean() * 100
print(success_rate_per_orbit)


Orbit
GTO            54.545455
HEO            50.000000
LEO            73.134328
MEO            66.666667
Polar          57.142857
Polar orbit     0.000000
SSO            85.714286
Sub-orbital     0.000000
Name: Class, dtype: float64


In [ ]:
ccafs_slc40_count = df[df['Launch site'] == 'CCAFS SLC 40'].shape[0]
print("Number of launches from CCAFS SLC 40:", ccafs_slc40_count)


Number of launches from CCAFS SLC 40: 0


In [ ]:
mask = df['Launch site'].str.contains(r'(CCAFS|CCSFS|Cape Canaveral).*SLC[-\s]?40',
                                      case=False, na=False)
ccafs_slc40_count = mask.sum()
print(ccafs_slc40_count)


53


C:\Users\whas3\AppData\Local\Temp\ipykernel_21764\1099838769.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['Launch site'].str.contains(r'(CCAFS|CCSFS|Cape Canaveral).*SLC[-\s]?40',


In [ ]:
print(df['Launch site'].value_counts(dropna=False))


Launch site
CCAFS , SLC-40             39
KSC , LC-39A               32
Cape Canaveral , LC-40     19
VAFB , SLC-4E              16
CCSFS , SLC-40             10
Cape Canaveral , SLC-40     1
CCAFS SLC-40                1
CCSFS , SLC-40 [ 597 ]      1
KSC , LC-39A [ 661 ]        1
CCSFS , SLC-40 [ 665 ]      1
Name: count, dtype: int64


In [ ]:
import re

def normalize_site(s):
    if pd.isna(s): return s
    t = s.lower()
    t = re.sub(r'\s+', ' ', t)
    # normalize common synonyms
    if ('slc' in t and ('40' in t)) and any(x in t for x in ['ccafs','ccsfs','cape canaveral']):
        return 'CCAFS SLC 40'
    if any(x in t for x in ['ksc','kennedy']) and ('39a' in t):
        return 'KSC LC 39A'
    if any(x in t for x in ['vafb','vandenberg']) and ('4e' in t):
        return 'VAFB SLC 4E'
    return s

df['site_norm'] = df['Launch site'].apply(normalize_site)

print(df['site_norm'].value_counts())
print('CCAFS SLC 40 count:', (df['site_norm'] == 'CCAFS SLC 40').sum())


site_norm
CCAFS SLC 40              53
KSC LC 39A                33
Cape Canaveral , LC-40    19
VAFB SLC 4E               16
Name: count, dtype: int64
CCAFS SLC 40 count: 53


In [ ]:
import re
import numpy as np
import pandas as pd

# 1) See a few raw values (optional)
print(df['Launch site'].value_counts().head(10))

# 2) Count all rows that are Cape Canaveral + pad 40 (any spelling)
mask_ccafs_slc40 = df['Launch site'].str.contains(
    r'(?:CCAFS|CCSFS|Cape\s+Canaveral).*(?:SLC|LC)[-\s]?40',
    case=False, na=False
)
ccafs_slc40_count = int(mask_ccafs_slc40.sum())
print("CCAFS SLC 40 launches:", ccafs_slc40_count)

# 3) (Optional) Create normalized columns so you can reuse later
df['pad'] = (
    df['Launch site']
      .str.extract(r'((?:SLC|LC)[-\s]?\d+[A-Z]?)', expand=False)
      .str.upper()
      .str.replace('-', ' ', regex=False)
)

def canonical_base(s):
    t = str(s).lower()
    if any(x in t for x in ['vafb','vandenberg']): return 'VAFB'
    if any(x in t for x in ['ksc','kennedy']):     return 'KSC'
    if any(x in t for x in ['ccafs','ccsfs','cape canaveral']): return 'Cape Canaveral'
    return s

df['site_base'] = df['Launch site'].apply(canonical_base)

# Example: count by (site_base, pad)
print(df.groupby(['site_base','pad']).size().sort_values(ascending=False))


Launch site
CCAFS , SLC-40             39
KSC , LC-39A               32
Cape Canaveral , LC-40     19
VAFB , SLC-4E              16
CCSFS , SLC-40             10
Cape Canaveral , SLC-40     1
CCAFS SLC-40                1
CCSFS , SLC-40 [ 597 ]      1
KSC , LC-39A [ 661 ]        1
CCSFS , SLC-40 [ 665 ]      1
Name: count, dtype: int64
CCAFS SLC 40 launches: 72
site_base       pad   
Cape Canaveral  SLC 40    53
KSC             LC 39A    33
Cape Canaveral  LC 40     19
VAFB            SLC 4E    16
dtype: int64


In [ ]:
success_rate = df['Class'].mean()
print("Success rate:", round(success_rate*100, 2), "%")


Success rate: 66.12 %


In [ ]:
import re
import pandas as pd

# clean refs like "[ 597 ]" and normalize whitespace
clean = (df['Launch site']
           .str.replace(r'\[\s*\d+\s*\]', '', regex=True)
           .str.replace(r'\s+', ' ', regex=True)
           .str.strip())

# base site
def base_site(s):
    s = s.lower()
    if 'vafb' in s or 'vandenberg' in s: return 'Vandenberg'
    if 'ksc' in s or 'kennedy' in s:     return 'Kennedy'
    if 'ccafs' in s or 'ccsfs' in s or 'cape canaveral' in s: return 'Cape Canaveral'
    return s

# pad (normalize LC/SLC and hyphen/space)
pad = (clean.str.extract(r'((?:SLC|LC)[-\s]?\d+[A-Z]?)', expand=False)
             .str.upper()
             .str.replace('-', ' ', regex=False))

site_base = clean.apply(base_site)

# count “Cape Canaveral + pad 40” (LC 40 or SLC 40)
ccafs_pad40 = (site_base.eq('Cape Canaveral') & pad.str.contains(r'\b40\b', na=False)).sum()
print("Cape Canaveral pad 40 launches:", int(ccafs_pad40))  # -> 72

# (optional) see the breakdown by pad label
print(pd.crosstab(site_base, pad).loc['Cape Canaveral', ['SLC 40','LC 40']])


Cape Canaveral pad 40 launches: 72
Launch site
SLC 40    53
LC 40     19
Name: Cape Canaveral, dtype: int64


In [ ]:
gto_count = (df['Orbit'] == 'GTO').sum()
print("Number of launches to geosynchronous orbit (GTO):", gto_count)


Number of launches to geosynchronous orbit (GTO): 33


In [ ]:
# Count successful landings on a drone ship (ASDS)
success_drone = df[(df['Class'] == 1) & (df['Booster landing'].str.contains('ASDS', case=False, na=False))]

print("Successful drone ship landings:", success_drone.shape[0])


Successful drone ship landings: 0


In [ ]:
# Count successful landings on a drone ship
success_drone = df[(df['Class'] == 1) & (df['Booster landing'].str.contains('drone', case=False, na=False))]

print("Successful drone ship landings:", success_drone.shape[0])


Successful drone ship landings: 64


In [ ]:
import re

# Peek at how the column is written in your file
print(df['Booster landing'].value_counts().head(15))

# Normalize spacing and case
bl = (df['Booster landing']
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip())

# Success flag (from your web-scraped labels)
is_success = bl.str.contains(r'\bsuccess\b', case=False, na=False)

# Drone-ship keywords used on Wikipedia
drone_keywords = r'(drone ship|ASDS|OCISLY|JRTI|ASOG|Of Course I Still Love You|Just Read the Instructions|A Shortfall of Gravitas)'

is_drone = bl.str.contains(drone_keywords, case=False, na=False)

# Count successful drone-ship landings
success_drone_count = int((is_success & is_drone).sum())
print("Successful drone-ship landings:", success_drone_count)


Booster landing
Success (drone ship)                 53
Success (ground pad)                 14
Failure (drone ship)                  6
No attempt [ 165 ]                    6
No attempt                            5
Controlled (ocean) [ d ] [ 46 ]       2
Failure [ 9 ] [ 10 ] (parachute)      1
Uncontrolled (ocean) [ d ]            1
Failure [ 9 ] [ 14 ] (parachute)      1
Uncontrolled (ocean) [ d ] [ 64 ]     1
No attempt [ 57 ]                     1
No attempt [ 43 ]                     1
No attempt [ 38 ]                     1
No attempt [ 89 ]                     1
Controlled (ocean) [ d ]              1
Name: count, dtype: int64
Successful drone-ship landings: 64


C:\Users\whas3\AppData\Local\Temp\ipykernel_21764\2945479242.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_drone = bl.str.contains(drone_keywords, case=False, na=False)


In [ ]:
def landing_status(td):
    out = [i for i in td.strings][0]   # <- only the FIRST token (e.g., "Success")
    return out


In [ ]:
def to_class(s):
    t = str(s).lower()
    return 1 if "success" in t and not any(x in t for x in ["partial","uncontrolled","failure"]) else 0

df['Class'] = df['Booster landing'].map(to_class).fillna(0).astype(int)


In [ ]:
bl = (df['Booster landing'].astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip())

drone_keywords = r'(drone ship|ASDS|OCISLY|JRTI|ASOG|Of Course I Still Love You|Just Read the Instructions|A Shortfall of Gravitas)'

success_drone_count = ((df['Class'].eq(1)) & bl.str.contains(drone_keywords, case=False, na=False)).sum()
print("Successful drone-ship landings:", int(success_drone_count))


C:\Users\whas3\AppData\Local\Temp\ipykernel_21764\902121265.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  success_drone_count = ((df['Class'].eq(1)) & bl.str.contains(drone_keywords, case=False, na=False)).sum()


Successful drone-ship landings: 64


In [ ]:
print(df['Booster landing'].value_counts().head(20))


Booster landing
Success (drone ship)                 53
Success (ground pad)                 14
Failure (drone ship)                  6
No attempt [ 165 ]                    6
No attempt                            5
Controlled (ocean) [ d ] [ 46 ]       2
Failure [ 9 ] [ 10 ] (parachute)      1
Uncontrolled (ocean) [ d ]            1
Failure [ 9 ] [ 14 ] (parachute)      1
Uncontrolled (ocean) [ d ] [ 64 ]     1
No attempt [ 57 ]                     1
No attempt [ 43 ]                     1
No attempt [ 38 ]                     1
No attempt [ 89 ]                     1
Controlled (ocean) [ d ]              1
No attempt [ 77 ]                     1
Failure [ 83 ] (drone ship)           1
Success [ 118 ] (drone ship)          1
Success [ 99 ] (ground pad)           1
Precluded [ 94 ] (drone ship)         1
Name: count, dtype: int64


In [ ]:
# clean refs like "[ 118 ]"
bl = (df['Booster landing'].astype(str)
        .str.replace(r'\[\s*\d+\s*\]', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip())

# success on a drone ship only
mask_success = bl.str.match(r'^Success\b', case=False)
mask_drone   = bl.str.contains(r'\(drone ship\)', case=False)

success_drone_count = int((mask_success & mask_drone).sum())
print("Successful drone-ship landings:", success_drone_count)  # -> 54


Successful drone-ship landings: 64


In [ ]:
# normalize launch site names
sites = (df['Launch site']
           .astype(str)
           .str.replace(r'\[.*?\]', '', regex=True)   # remove bracketed refs
           .str.replace(r'\s+', ' ', regex=True)      # normalize spaces
           .str.strip())

# mask for pad 40 at Cape Canaveral
mask_ccafs40 = sites.str.contains(r'(CCAFS|CCSFS|Cape Canaveral).*(LC|SLC)[ -]?40',
                                  case=False, na=False)

ccafs40_count = int(mask_ccafs40.sum())
print("CCAFS SLC-40 launches:", ccafs40_count)

# optional: show breakdown
print(sites[mask_ccafs40].value_counts())


CCAFS SLC-40 launches: 72
Launch site
CCAFS , SLC-40             39
Cape Canaveral , LC-40     19
CCSFS , SLC-40             12
Cape Canaveral , SLC-40     1
CCAFS SLC-40                1
Name: count, dtype: int64


C:\Users\whas3\AppData\Local\Temp\ipykernel_21764\3286520475.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_ccafs40 = sites.str.contains(r'(CCAFS|CCSFS|Cape Canaveral).*(LC|SLC)[ -]?40',
